# A CNN-LSTM framework for the solar wind density forecasting
In this notebook we pre-process solar wind and solar binary mask for the time series analysis


## Notebook Contributors
* Andrea Giuseppe Di Francesco -- email: difrancesco.1836928@studenti.uniroma1.it
* Massimo Coppotelli -- email: coppotelli.1705325@studenti.uniroma1.it

In [ ]:
# !pip install pandas
# !pip install numpy
# !pip install torch

In [2]:
import pandas as pd
import numpy as np
import torch


/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# with open('./wind-2012-20221014T050711Z-001/wind-2012/WI_ELM2_3DP_45129.csv', 'r') as f:
#     skips = 0
#     for line in f.readlines():
#         # print(line)
#         if line.startswith("#"):
#             skips += 1
#         else:
#             break

proton_density_data = pd.read_csv('./wind-2012-20221014T050711Z-001/wind-2012/WI_ELM2_3DP_45129.csv',  comment = '#')

proton_density_data

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,EPOCH_yyyy-mm-ddThh:mm:ss.sssZ,ELECTRON_NO_DENSITY_
2012-01-01T00:03:00.000Z,6.748405,1.849119e-01,1.849119e-01,2004.1908,1.072296e+01,1.072296e+01,403.15607,2.219659e+00,2.219659e+00,1.325376e+09,1.266896e+02,1.266896e+02,4.0,4.0,4.0,4.0,-400.27461,42.257462,-22.531933,2.047825e+00,5.110201e+00,1.872841e+00,2.047825e+00,5.110201e+00,1.872841e+00,-270125520.0,28479028.0,-15181566.0,7.717828e+06,3.174971e+06,8.708165e+05,7.717828e+06,3.174971e+06,8.708165e+05,4.0,4.0,4.0,4.0,4.0,4.0
2012-01-01T00:09:00.000Z,7.009005,5.947632e-02,5.947632e-02,1974.1769,2.449270e+00,2.449270e+00,406.56082,6.935384e+00,6.935384e+00,1.325377e+09,9.813333e+01,9.813333e+01,3.0,3.0,3.0,3.0,-404.26935,38.545035,-19.231332,7.020858e+00,8.510507e-01,1.315627e+00,7.020858e+00,8.510507e-01,1.315627e+00,-283362040.0,27013260.0,-13479277.0,6.143970e+06,4.120637e+05,9.286992e+05,6.143970e+06,4.120637e+05,9.286992e+05,3.0,3.0,3.0,3.0,3.0,3.0
2012-01-01T00:15:00.000Z,6.309707,1.999256e-01,1.999256e-01,1988.2557,7.922371e+00,7.922371e+00,392.01508,5.208294e+00,5.208294e+00,1.325377e+09,1.267462e+02,1.267462e+02,4.0,4.0,4.0,4.0,-390.48546,30.037333,-16.002583,5.218107e+00,4.631828e+00,5.469909e+00,5.218107e+00,4.631828e+00,5.469909e+00,-246371840.0,18896130.0,-10147033.0,8.005556e+06,2.481644e+06,3.679440e+06,8.005556e+06,2.481644e+06,3.679440e+06,4.0,4.0,4.0,4.0,4.0,4.0
2012-01-01T00:21:00.000Z,6.925176,1.742743e-01,1.742743e-01,1972.8583,6.167188e+00,6.167188e+00,411.12383,1.089194e+01,1.089194e+01,1.325377e+09,1.267462e+02,1.267462e+02,4.0,4.0,4.0,4.0,-409.09447,34.257119,-22.009475,1.058593e+01,3.234491e+00,2.200508e+00,1.058593e+01,3.234491e+00,2.200508e+00,-283214320.0,23696171.0,-15235759.0,5.938361e+06,1.912239e+06,1.457713e+06,5.938361e+06,1.912239e+06,1.457713e+06,4.0,4.0,4.0,4.0,4.0,4.0
2012-01-01T00:27:00.000Z,6.120157,-1.000000e+31,-1.000000e+31,1997.1027,-1.000000e+31,-1.000000e+31,402.34685,-1.000000e+31,-1.000000e+31,1.325378e+09,-1.000000e+31,-1.000000e+31,2.0,2.0,2.0,2.0,-401.16205,26.704665,-15.394300,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-245562500.0,16319764.0,-9426279.0,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-31T23:33:00.000Z,1.161837,1.934753e-02,1.934753e-02,2362.8966,2.355405e+01,2.355405e+01,420.68945,1.878427e+01,1.878427e+01,1.356997e+09,9.823837e+01,9.823837e+01,3.0,3.0,3.0,3.0,-417.62873,39.885293,-29.876685,1.771486e+01,1.268165e+01,1.344711e+00,1.771486e+01,1.268165e+01,1.344711e+00,-48502260.0,4620211.3,-3472917.1,1.418672e+06,1.394793e+06,2.135721e+05,1.418672e+06,1.394793e+06,2.135721e+05,3.0,3.0,3.0,3.0,3.0,3.0
2012-12-31T23:39:00.000Z,1.159067,1.766642e-02,1.766642e-02,2371.6412,2.939394e+01,2.939394e+01,430.75958,8.072315e+00,8.072315e+00,1.356997e+09,1.268252e+02,1.268252e+02,4.0,4.0,4.0,4.0,-427.15110,46.960228,-29.428394,7.847089e+00,4.855296e+00,3.553061e+00,7.847089e+00,4.855296e+00,3.553061e+00,-49505669.0,5446227.9,-3409042.7,9.266923e+05,6.109780e+05,3.907505e+05,9.266923e+05,6.109780e+05,3.907505e+05,4.0,4.0,4.0,4.0,4.0,4.0
2012-12-31T23:45:00.000Z,1.245448,1.994223e-02,1.994223e-02,2333.1077,3.744924e+01,3.744924e+01,443.58084,6.740573e+00,6.740573e+00,1.356998e+09,1.268252e+02,1.268252e+02,4.0,4.0,4.0,4.0,-438.54845,59.733465,-28.453996,6.355240e+00,6.985666e+00,6.186220e+00,6.355240e+00,6.985666e+00,6.186220e+00,-54627525.0,7443722.0,-3536517.8,1.618015e+06,9.272400e+05,7.341923e+05,1.618015e+06,9.272400e+05,7.341923e+05,4.0,4.0,4.0,4.0,4.0,4.0
2012-12-31T23:51:00.000Z,1.276327,2.962419e-02,2.962419e-02,2322.7836,4.419419e+01,4.419419e+01,451.76591,2.946918e+01,2.946918e+01,1.356998e+09,9.823837e+01,9.823837e+01,3.0,3.0,3.0,3.0,-446.81561,60.858833,-25.972604,2.796099e+01,9.944018e+00,9.619840e+